In [ ]:
from matplotlib import pyplot as plt

import numpy as np
import cv2 as cv

In [ ]:
filename = './videos/gray/news.avi'

In [ ]:
def diff_pixels(frameA, frameB, params):
    diff = np.abs(frameA - frameB)
    return np.where(diff > params[0], 1, 0).sum()


In [ ]:
def diff_blocks(frameA, frameB, params):
    diff = np.abs(frameA - frameB)
    diff = diff * diff

    count = 0
    for row in range(0, frameA.shape[0], params[0]):
        for col in range(0, frameA.shape[1], params[0]):
            root = np.sqrt(diff[row:row+params[0], col:col+params[0]].sum())
            if root > params[1]:
                count += 1

    return count


In [ ]:
def normalize(img_in):
    img_out = np.zeros(img_in.shape, dtype="uint8")

    tmin = img_in.min()
    tmax = img_in.max()

    tsize = abs(tmax - tmin)
    img_out = ((img_in - tmin) * 255) // tsize
    return img_out


def sobelX(img_in):
    res_h = img_in[:, 2:] - img_in[:, :-2]
    res_v = res_h[:-2] + res_h[2:] + 2*res_h[1:-1]
    return np.abs(res_v)


def sobelY(img_in):
    img = img_in.transpose()
    res_h = img[:, 2:] - img[:, :-2]
    res_v = res_h[:-2] + res_h[2:] + 2*res_h[1:-1]
    return np.abs(res_v.transpose())


def sobel(img):
    img = img.astype('int16')
    out = np.zeros(img.shape, int)
    x = sobelX(img)
    y = sobelY(img)

    out[1:-1, 1:-1] = x + y

    return out


def diff_edges(frameA, frameB, params):
    maskA = normalize(sobel(frameA))
    return (maskA > 128).sum()


In [ ]:
def diff_histogram(frameA, frameB, params):
    histA, _ = np.histogram(frameA, bins=256, range=(0, 255))
    histB, _ = np.histogram(frameB, bins=256, range=(0, 255))

    return np.abs(histA-histB).sum()


In [ ]:
def get_metrics(file, difference, params, dtype='int16'):
    cap = cv.VideoCapture(file)

    metrics = []

    ret, frame = cap.read()

    old_frame = frame[:, :, 0].astype(dtype)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            gray_frame = frame[:, :, 0].astype(dtype)
            metrics.append(difference(gray_frame, old_frame, params))
            old_frame = gray_frame
        else:
            cap.release()
            break

    return metrics


In [ ]:
Y = np.array(get_metrics(filename, diff_pixels, [128]))
X = range(len(Y))

plt.figure(figsize=(25,10))
plt.plot(X, Y)

In [ ]:
Y = np.array(get_metrics(filename,diff_blocks, [8, 64, 128], 'int32'))
X = range(len(Y))

plt.figure(figsize=(25, 10))
plt.plot(X, Y)

In [ ]:
Y = np.array(get_metrics(filename,diff_histogram, []))
X = range(len(Y))

plt.figure(figsize=(25, 10))
plt.plot(X, Y)

In [ ]:
Y = np.array(get_metrics(filename, diff_edges, []))
X = range(len(Y))

plt.figure(figsize=(25, 10))
plt.plot(X, Y)
